## Playground season 3, episode 07
### Start: February 14, 2023
### End: February 27, 2023

In [3]:
import pandas as pd
import numpy as np
import os

In [ ]:
if os.name == 'nt':
    data_path = r"D:\Coding_pratice\_Data\kaggle_playground_series\playground-series-s3e7"
else:
    data_path = ''
train_csv = os.path.join(data_path, 'train.csv')
test_csv = os.path.join(data_path, 'test.csv')